# Power Point Slide 생성

# 환경 설정

#### 참조:  Bedrock 호출
- 참고: https://docs.anthropic.com/en/api/claude-on-amazon-bedrock


In [1]:
# pip install boto3

In [2]:
%load_ext autoreload
%autoreload 2

# 시도1: Prompt Chaining

- 목표: 최소한의 API 호출로 Chaining Prompt를 사용하여 20장의 파워포인트 슬라이드를 생성하기
- 아이디어: 파워포인트 슬라이드 20장을 구성하기 위해 3번의 Bedrock invoke_model API를 호출한다. 
    - (1) Initial prompt를 사용해 전체 아웃라인 짜기 - 즉, 1번째 Bedrock invoke_model API를 호출에서는 토픽에 맞는 전체 개요를 만든다.
    - (2) Slide prompt를 사용해 1번에서 짠 아웃라인에 맞춰서 디테일한 내용을 불렛 포인트로 정리한 슬라이드를 각각 만든다 - 즉, 2번째 호출에서는 1~10번째 슬라이드의 개요를, 3번째 호출에서는 11~20번째 슬라이드 개요만을 사용해 각 슬라이드에 대한 내용을 자세하게 생성한다.

##  1. Outline prompt 준비

### 프롬프트 템플릿 파일 기술

In [3]:
from util.prompt_util import *

# 템플릿 파일의 경로를 지정합니다.
prompt_folder = "prompt_template"
outline_prompt_file_name = "v01_outline_prompt_template.txt"

### Outline prompt Template 로딩 및 토픽 추가

In [4]:
from jinja2 import Template

prompt_file_path = os.path.join(prompt_folder, outline_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
outline_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)
# print_json(prompt)
print(outline_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Analyze the given topic thoroughly to write in the provided <output_format> JSON format.
Return only the JSON object without additional tags or explanations.

<context>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</context>

<instruction>
1. Title:
   - Create a compelling and concise main title that captures the essence of the presentation.

2. Slides:
   - Generate 20 slides, including an opening and a closing slide.
   - For each slide:
     - Assign a slide_number (01 to 20).
     - Create a clear and informative slide_title.
     
3. Content Guidelines:
   - Ensure a logical flow and structure throughout the presentation.
   - Include an introduction slide (slide 02) and a conclusion slide (slide 19).   

4. Common Points:
   - Include the most important ke

### Outline prompt 실행

In [5]:
outline, input_tokens, output_tokens = invoke_claude(outline_prompt)
print("\n## Overall Outline by Claude Sonnet3.5:\n")
print(outline)
print(f"\n Input tokens: {input_tokens}, Output tokens: {output_tokens}")
print("\n" + "="*50 + "\n")




## Overall Outline by Claude Sonnet3.5:

{
    "title": "2024 파리 올림픽 폐막식 성공 예측: 글로벌 축제의 대미",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐막식 성공 예측"
        },
        {
            "slide_number": "02",
            "slide_title": "발표 개요 및 목적"
        },
        {
            "slide_number": "03",
            "slide_title": "2024 파리 올림픽 개요"
        },
        {
            "slide_number": "04",
            "slide_title": "올림픽 폐막식의 의미와 중요성"
        },
        {
            "slide_number": "05",
            "slide_title": "과거 올림픽 폐막식 성공 사례 분석"
        },
        {
            "slide_number": "06",
            "slide_title": "파리의 문화적 특성과 폐막식 연계 방안"
        },
        {
            "slide_number": "07",
            "slide_title": "폐막식 주요 행사 및 프로그램 구성"
        },
        {
            "slide_number": "08",
            "slide_title": "기술 혁신: AR, VR을 활용한 관객 경험 향상"
        },
        {
            "slide_number": "09",
            "slide_

### 슬라이드 타이틀만 보기

In [6]:
print_slide_titles(outline)

프레젠테이션 제목: 2024 파리 올림픽 폐막식 성공 예측: 글로벌 축제의 대미

슬라이드 01: 2024 파리 올림픽 폐막식 성공 예측
슬라이드 02: 발표 개요 및 목적
슬라이드 03: 2024 파리 올림픽 개요
슬라이드 04: 올림픽 폐막식의 의미와 중요성
슬라이드 05: 과거 올림픽 폐막식 성공 사례 분석
슬라이드 06: 파리의 문화적 특성과 폐막식 연계 방안
슬라이드 07: 폐막식 주요 행사 및 프로그램 구성
슬라이드 08: 기술 혁신: AR, VR을 활용한 관객 경험 향상
슬라이드 09: 지속가능성: 친환경 폐막식 계획
슬라이드 10: 글로벌 참여: 온라인 플랫폼 활용 전략
슬라이드 11: 안전 및 보안 대책
슬라이드 12: 폐막식 예산 분석 및 경제적 효과
슬라이드 13: 미디어 커버리지 및 글로벌 홍보 전략
슬라이드 14: 문화적 다양성 반영: 국제 협력 프로그램
슬라이드 15: 장애인 올림픽과의 연계성 강화
슬라이드 16: 폐막식 성공을 위한 주요 도전 과제
슬라이드 17: 성공 지표 및 평가 기준
슬라이드 18: 폐막식 이후: 레거시 프로그램
슬라이드 19: 결론: 2024 파리 올림픽 폐막식 성공 전망
슬라이드 20: 질의응답 및 토론


## 2. Slide prompt (각 슬라이드에 대한 프롬프트) 생성

In [7]:
# prompt_file_name = "v01_slide_prompt_template.txt"
# prompt_file_name = "v02_slide_prompt_template.txt"
prompt_file_name = "v03_slide_prompt_template.txt"

slide_prompt_file_path = os.path.join(prompt_folder, prompt_file_name)

template_content = load_template_from_file(slide_prompt_file_path)
slide_prompt_template = Template(template_content)
slide_prompt_template.render()

'You are an expert in creating impactful and professional PowerPoint presentations.\nYour task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.\n\nBased on the information, create detailed content for the specified slide:\n\n<topic>\n\n</topic>\n\n<outline>\n\n</outline>\n\n<slide_title>\n. \n</slide_title>\n\nGuidelines:\n1. Provide detailed content for the specified slide in Korean.\n2. Include 3-5 key bullet points that are directly relevant to the slide title and topic.\n3. Ensure there is no duplication of contnent from other slides in the presentation.\n4. Suggest an appropriate visual element (e.g., image, chart, graph) that enhances the understanding of the content.\n5. Add brief presenter notes to provide context or additional information.\n6. Use clear, concise language suitable for a professional presentation.\n7. Maintain consistency with the overall flow and narrative of 

## 추가 2번의 호출을 통해 슬라이드 생성
- 1번째 호출에서 1~10번째 슬라이드 생성
- 2번째 호출에서 11~20번째 슬라이드 생성

### 테스트1: 10장을 1~5번째, 6~10번째 슬라이드로 두번에 나눠서 만들면 -> 성공

In [8]:
from util.prompt_util import create_slide_prompt, create_part_slide_prompt
topic = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
# include_outline = True
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)

include_outline = False
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)
slide_prompt_list = create_part_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, n_slide = 5, max_slide_num=10)



## 1th call's slide_prompt: 
 You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on the information, create detailed content for the specified slide:

<topic>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</topic>

<outline>
Not Available
</outline>

<slide_title>
['01', '02', '03', '04', '05']. ['2024 파리 올림픽 폐막식 성공 예측', '발표 개요 및 목적', '2024 파리 올림픽 개요', '올림픽 폐막식의 의미와 중요성', '과거 올림픽 폐막식 성공 사례 분석']
</slide_title>

Guidelines:
1. Provide detailed content for the specified slide in Korean.
2. Include 3-5 key bullet points that are directly relevant to the slide title and topic.
3. Ensure there is no duplication of contnent from other slides in the presentation.
4. Suggest an appropriate visual element (e.g., image, chart, graph) that enhances the understanding of the content.
5. Add 

In [ ]:
slide_prompt_list

In [9]:
# 결과 폴더 생성

from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

In [10]:
# 슬라이드 생성

from util.prompt_util import generate_slide_content
generate_slide_content(slide_prompt_list, output_directory, include_outline)

1번째 호출 시작합니다.

1번째 호출: Input tokens: 657, Output tokens: 499 

2번째 호출 시작합니다.

2번째 호출: Input tokens: 709, Output tokens: 470 


 Detailed Slide Contents:
<slide_['01', '02', '03', '04', '05']>
제목: 2024 파리 올림픽 폐막식 성공 예측

내용:
• 성공적인 폐막식을 위한 핵심 요소 분석
• 파리의 문화적 특성과 올림픽 정신의 조화
• 기술 혁신과 전통의 융합을 통한 새로운 경험 창출
• 지속 가능성과 환경 친화적 접근 방식 강조
• 글로벌 시청자 참여 증대를 위한 전략

시각 요소: 파리의 상징적인 랜드마크(에펠탑, 개선문 등)와 올림픽 로고가 결합된 이미지. 중앙에는 성공 예측을 나타내는 상승 그래프 배치.

발표자 노트: 2024 파리 올림픽 폐막식의 성공을 예측하는 이 슬라이드는 전체 프레젠테이션의 핵심을 담고 있습니다. 파리의 독특한 문화적 특성과 올림픽 정신의 조화, 최신 기술의 활용, 환경 친화적 접근, 그리고 글로벌 참여 증대 전략 등 다양한 측면에서 폐막식의 성공 가능성을 분석합니다. 이어지는 슬라이드들에서 각 요소에 대해 더 자세히 다룰 예정입니다.
</slide_['01', '02', '03', '04', '05']>

--------------------------------------------------

<slide_['06', '07', '08', '09', '10']>
제목: 파리의 문화적 특성과 폐막식 연계 방안

내용:
• 에펠탑을 활용한 대규모 라이트쇼 계획
• 프랑스 예술과 패션을 결합한 퍼포먼스 구성
• 파리의 역사적 랜드마크를 배경으로 한 폐막식 진행
• 프랑스 전통 음악과 현대 음악의 조화로운 융합
• 세계적인 프랑스 요리사들이 참여하는 특별 만찬 이벤트

시각 요소: 파리의 주요 랜드마크(에펠탑, 루브르 박물관, 개선문 등)를 배경으로 한 폐막식 무대 콘셉트 이미지


### 테스트 2: 12장의 슬라이드를 1~6번째, 7~12번째로 나눠서 만들면 -> 성공

In [24]:
from util.prompt_util import create_slide_prompt, create_part_slide_prompt
topic = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
# include_outline = True
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)

include_outline = False
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)
slide_prompt_list = create_part_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, n_slide = 6, max_slide_num=12)



## 1th call's slide_prompt: 
 You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on the following outline for a presentation on topic, 
create detailed content for slide_title on slide number.
Include bullet points and any relevant information that should be on this slide.
But, there should be no duplicated content between slides.
Now, provide detailed content for the slide.

<topic>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</topic>

<outline>
Not Available
</outline>

<slide_number>
['01', '02', '03', '04', '05', '06']
</slide_number>

<slide_title>
['2024 파리 올림픽 폐회식 성공 예측 분석', '소개: 파리 올림픽의 의의와 폐회식의 중요성', '2024 파리 올림픽 개요 및 주요 특징', '역대 올림픽 폐회식 성공 사례 분석', '파리 2024 조직위원회의 폐회식 계획', '폐회식 장소 및 인프라 준비 현황']
</slide_title>


## 2th call's slide_prompt: 
 You are an expert in creating impactful and professional 

In [25]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

In [26]:
from util.prompt_util import generate_slide_content
generate_slide_content(slide_prompt_list, output_directory, include_outline)

각 슬라이드에 대한 상세 내용을 한국어로 제공하겠습니다.

슬라이드 01: 2024 파리 올림픽 폐회식 성공 예측 분석
• 발표 제목: "2024 파리 올림픽 폐회식 성공 예측 분석"
• 발표자 이름 및 소속
• 발표 일자
• 주요 내용 미리보기:
  - 파리 올림픽의 의의
  - 폐회식의 중요성
  - 성공 요인 분석
  - 예측 및 전망

슬라이드 02: 소개: 파리 올림픽의 의의와 폐회식의 중요성
• 파리 올림픽의 역사적 의의:
  - 1900년, 1924년에 이어 세 번째 개최
  - 100년 만의 파리 올림픽 개최
• 올림픽 폐회식의 중요성:
  - 대회의 마지막을 장식하는 행사
  - 개최국의 문화와 성과를 세계에 알리는 기회
  - 다음 올림픽 개최국으로의 연결 고리

슬라이드 03: 2024 파리 올림픽 개요 및 주요 특징
• 개최 기간: 2024년 7월 26일 - 8월 11일
• 참가국 수: 예상 206개국
• 종목 수: 32개 종목, 329개 세부 경기
• 주요 특징:
  - 지속 가능성 강조
  - 성평등 실현 목표
  - 도시 랜드마크를 활용한 경기장 배치

슬라이드 04: 역대 올림픽 폐회식 성공 사례 분석
• 2012 런던 올림픽:
  - 영국 대중문화의 축제
  - 세계적 아티스트들의 공연
• 2016 리우 올림픽:
  - 삼바와 카니발 문화 강조
  - 환경 보호 메시지 전달
• 2020 도쿄 올림픽:
  - 코로나19 상황에서의 혁신적인 접근
  - 전통과 현대의 조화

슬라이드 05: 파리 2024 조직위원회의 폐회식 계획
• 테마: "파리의 예술과
 혁신의 조화"
• 주요 계획:
  - 세느강을 활용한 수상 퍼레이드
  - 프랑스 예술과 문화의 집약적 쇼케이스
  - 친환경 요소 강조
  - 디지털 기술을 활용한 관객 참여 유도

슬라이드 06: 폐회식 장소 및 인프라 준비 현황
• 메인 장소: 트로카데로 광장
  - 수용 인원: 약 100,000명
  - 에펠탑을 배경으로 한 장관 연출
• 보조 장소: 세느강 주변
  - 강

### 테스트 3: 16장의 슬라이드를 1~8번째, 9~16번째로 나눠서 만들면 -> 성공

In [27]:
from util.prompt_util import create_slide_prompt, create_part_slide_prompt
topic = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
# include_outline = True
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)

include_outline = False
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)
slide_prompt_list = create_part_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, n_slide = 8, max_slide_num=16)



## 1th call's slide_prompt: 
 You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on the following outline for a presentation on topic, 
create detailed content for slide_title on slide number.
Include bullet points and any relevant information that should be on this slide.
But, there should be no duplicated content between slides.
Now, provide detailed content for the slide.

<topic>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</topic>

<outline>
Not Available
</outline>

<slide_number>
['01', '02', '03', '04', '05', '06', '07', '08']
</slide_number>

<slide_title>
['2024 파리 올림픽 폐회식 성공 예측 분석', '소개: 파리 올림픽의 의의와 폐회식의 중요성', '2024 파리 올림픽 개요 및 주요 특징', '역대 올림픽 폐회식 성공 사례 분석', '파리 2024 조직위원회의 폐회식 계획', '폐회식 장소 및 인프라 준비 현황', '예상 참가국 및 선수단 규모', '폐회식 예술 공연 및 문화 프로그램']
</slide_title>


## 2th call's slide_prompt: 
 You

In [28]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

In [30]:
from util.prompt_util import generate_slide_content
generate_slide_content(slide_prompt_list, output_directory, include_outline)

주어진 정보를 바탕으로 각 슬라이드의 상세 내용을 한국어로 작성하겠습니다.

슬라이드 1: 2024 파리 올림픽 폐회식 성공 예측 분석

• 발표 주제 소개
• 분석의 목적과 중요성
• 주요 분석 포인트 개요
  - 역사적 맥락
  - 현재 준비 상황
  - 예상되는 도전과 기회
• 발표 구조 안내

슬라이드 2: 소개: 파리 올림픽의 의의와 폐회식의 중요성

• 파리 올림픽의 역사적 의미
  - 1900년, 1924년에 이은 세 번째 개최
• 올림픽 폐회식의 상징성
  - 평화와 화합의 메시지 전달
  - 개최국 문화의 세계적 홍보 기회
• 폐회식이 올림픽 전체 평가에 미치는 영향
• 2024 파리 올림픽 폐회식의 특별한 의미

슬라이드 3: 2024 파리 올림픽 개요 및 주요 특징

• 개최 기간: 2024년 7월 26일 ~ 8월 11일
• 주요 경기장 소개
• 새롭게 도입되는 종목
• 파리 올림픽의 슬로건 및 로고 소개
• 지속가능성에 중점을 둔 친환경 올림픽 계획

슬라이드 4: 역대 올림픽 폐회식 성공 사례 분석

• 2012 런던 올림픽 폐회식
  - 영국 대중문화의 성공적 홍보
• 2016 리우 올림픽 폐회식
  - 브라질 전통과 현대의 조화
• 2020 도쿄 올림픽 폐회식
  - 팬데믹 상황에서의 혁신적 접근
• 성공적인 폐회식의 공통 요소 분석

슬라이드 5: 파리 2024 조직위원회의 폐회식 계획

• 폐회식 기본 컨셉 및 주제
• 주요 프로그램 구성 요소
• 혁신적인 기술 활용 계획
  - AR/VR 기술 통합
  - 환경 친화적 요소 도입
• 세계적 아티스트 및 유명 인사 참여 계획

슬라이드 6: 폐회식 장소 및 인프라 준비 현황

• 폐회식 주 무대: 스타드 드 프랑스
  - 수용 인원 및 시설 현황
• 보조 행사장 및 주변 인프라
• 교통 및 숙박 시설 준비 상황
• 안전 및 보안 대책
• 코로나19 대응 계획

슬라이드 7: 예상 참가국 및 선수단 규모

• 예상 참가국 수: 약 200개국
• 예상 선수단 규모: 약 10,500명


### 테스트 4: 18장의 슬라이드를 1~9번째, 10~18번째로 나눠서 만들면 -> 1번째만 만들어지고, 그 다음 10~18번째 슬라이드가 만들어짐
- 1~9번째 슬라이드를 만드는 rate limit throttling이 걸린 게 아닐까 싶어서 prompt_utils.py에 invoke_claude 함수에 sleep(80) -> sleep(120)을 주어도 현상은 똑같음

In [26]:
from util.prompt_util import create_slide_prompt, create_part_slide_prompt
topic = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
# include_outline = True
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)

include_outline = False
# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)
slide_prompt_list = create_part_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, n_slide = 9, max_slide_num=18)



## 1th call's slide_prompt: 
 You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on the information, create detailed content for the specified slide:

<topic>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</topic>

<outline>
Not Available
</outline>

<slide_title>
['01', '02', '03', '04', '05', '06', '07', '08', '09'].['2024 파리 올림픽 폐회식 성공 예측', '서론: 올림픽 폐회식의 중요성', '파리 2024 올림픽 개요', '과거 올림픽 폐회식 성공 사례 분석', '파리의 문화적 특성과 폐회식 컨셉', '폐회식 장소 및 시설 준비 현황', '기술적 혁신: AR, VR 활용 계획', '환경 친화적 요소 도입 전략', '참가국 및 선수단 퍼레이드 계획']
</slide_title>

Guidelines:
1. Provide detailed content for the specified slide in Korean.
2. Include 3-5 key bullet points that are directly relevant to the slide title and topic.
3. Ensure there is no duplication of contnent from other slides in the presentation.
4. Sugge

In [27]:
slide_prompt_list

["You are an expert in creating impactful and professional PowerPoint presentations.\nYour task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.\n\nBased on the information, create detailed content for the specified slide:\n\n<topic>\nParis 2024 Olympics Cloasing Ceremony Success Prediction\n</topic>\n\n<outline>\nNot Available\n</outline>\n\n<slide_title>\n['01', '02', '03', '04', '05', '06', '07', '08', '09'].['2024 파리 올림픽 폐회식 성공 예측', '서론: 올림픽 폐회식의 중요성', '파리 2024 올림픽 개요', '과거 올림픽 폐회식 성공 사례 분석', '파리의 문화적 특성과 폐회식 컨셉', '폐회식 장소 및 시설 준비 현황', '기술적 혁신: AR, VR 활용 계획', '환경 친화적 요소 도입 전략', '참가국 및 선수단 퍼레이드 계획']\n</slide_title>\n\nGuidelines:\n1. Provide detailed content for the specified slide in Korean.\n2. Include 3-5 key bullet points that are directly relevant to the slide title and topic.\n3. Ensure there is no duplication of contnent from other slides in the presentation.\n4. Suggest an a

In [28]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

In [30]:
from util.prompt_util import generate_slide_content
generate_slide_content(slide_prompt_list, output_directory, include_outline)

1번째 호출: Input tokens: 871, Output tokens: 416 

2번째 호출 시작합니다.

2번째 호출: Input tokens: 863, Output tokens: 497 


--- Generated Slides ---

Slide 1:
<slide_01>
제목: 2024 파리 올림픽 폐회식 성공 예측

내용:
• 파리 2024 올림픽 폐회식의 주요 성공 요인 분석
• 문화적, 기술적, 환경적 측면에서의 혁신 예상
• 과거 올림픽 대비 예상되는 차별화 포인트
• 글로벌 시청자 수 및 참여도 예측
• 경제적 파급 효과 전망

시각 요소: 파리 2024 올림픽 로고와 에펠탑을 배경으로 한 폐회식 예상 이미지 몽타주

발표자 노트: 이 슬라이드는 프레젠테이션의 전체적인 방향을 설정합니다. 파리 2024 올림픽 폐회식의 성공을 예측하는 주요 요소들을 간략히 소개하고, 이어지는 슬라이드에서 각 요소를 자세히 다룰 것임을 언급하세요. 특히 파리의 문화적 특성과 최신 기술의 결합이 어떻게 독특한 폐회식을 만들어낼지에 대한 기대감을 조성하는 것이 중요합니다.
</slide_01>

----------------------------------------

Slide 2:
<slide_10>
제목: 문화 공연 및 예술 프로그램 구성

내용:
• 프랑스 문화 유산을 반영한 공연 구성: 에펠탑, 루브르 등 상징적 장소 활용
• 현대 예술과 전통 예술의 조화: 발레, 현대무용, 서커스 등 다양한 장르 융합
• 국제적 아티스트 참여: 세계적인 가수, 댄서, 뮤지션 협업으로 글로벌 화합 강조
• 디지털 기술을 활용한 혁신적 무대 연출: AR, VR 기술 도입으로 관객 몰입도 증대
• 지속가능성 메시지 전달: 환경 보호와 평화를 주제로 한 예술 작품 전시

시각 요소: 다양한 문화 공연 장면을 콜라주 형식으로 구성한 이미지, 주요 공연 장소와 아티스트 정보를 포함한 인포그래픽

발표자 노트: 파리의 문화적 다양성과 예술적 풍부함을 강조하며, 올림픽의 가치와 프랑

# 시도2: Zero shot prompting 이지만 Role-based prompting, Task-specific instructions, Content structuring, Output constraints를 사용
: Structure the presentation을 미리 prompt (presentation_prompt) 에 써주고, 각 슬라이드의 아웃풋 토큰 개수에 제한을 두기

### 프롬프트 로딩

In [37]:
all_prompt_file_name = "v01_all_prompt_template.txt"

prompt_file_path = os.path.join(prompt_folder, all_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
presentation_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)
# print_json(prompt)
print(presentation_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, 
ensuring a logical flow and comprehensive coverage of the topic.

<topic>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</topic>

<instruction>
For each slide, provide:
1. A clear and concise title
2. 3-5 bullet points of key information
3. Any relevant data or statistics (if applicable)

Structure the presentation as follows:
- Slide 1: Introduction
- Slides 2-3: Background and Context
- Slides 4-7: Main Points and Analysis
- Slides 8-9: Predictions and Potential Outcomes
- Slide 10: Conclusion and Key Takeaways

Present the content for all 20 slides in a clear, organized manner. 
The number of output tokens for each slide must not exceed 100
</instruction>


### 슬라이드 생성

In [43]:
from util.prompt_util import generate_all_slide_content

generate_all_slide_content(presentation_prompt, output_directory)

Generated Presentation Content:
Here's a detailed outline for a 20-slide presentation on "Paris 2024 Olympics Closing Ceremony Success Prediction" in Korean:

1. 제목: 파리 2024 올림픽 폐회식 성공 예측
• 발표 주제 소개
• 발표자 소개
• 주요 논점 요약

2. 올림픽 폐회식의 중요성
• 올림픽 정신 구현
• 개최국 문화 홍보
• 전 세계 시청자 유치
• 경제적 영향

3. 과거 올림픽 폐회식 분석
• 런던 2012: 8,000만 시청자
• 리우 2016: 3,500만 시청자
• 도쿄 2020: 코로나19로 인한 제한적 행사

4. 파리 2024 올림픽 개요
• 개최 일정: 2024년 7월 26일 - 8월 11일
• 주요 경기장: 에펠탑, 베르사유 궁전 등
• 예상 참가국: 206개국
• 예상 관람객: 1,300만 명

5. 파리의 문화적 강점
• 세계적인 관광 도시
• 풍부한 예술 및 문화 유산
• 패션과 요리의 중심지
• 혁신적인 기술 도시

6. 폐회식 장소: 세느 강
• 독특하고 혁신적인 선택
• 파리의 상징적 장소
• 16km 구간 활용
• 예상 현장 관객: 60만 명

7. 기술 혁신 예측
• 증강현실(AR) 활용
• 드론 쇼
• 실시간 다국어 통역 서비스
• 5G 기반 실시간 중계

8. 환경 친화적 접근
• 재생 에너지 100% 사용 목표
• 탄소 중립 행사 계획
• 재활용 소재 활용
• 대중교통 이용 촉진

9. 글로벌 참여 전략
• 소셜 미디어 캠페인
• 실시간 투표 및 상호작용
• 가상현실(VR) 체험 제공
• 다국적 아티스트 참여

10. 결론 및 주요 시사점
• 혁신적이고 지속 가능한 폐회식 예상
• 높은 시청률 및 글로벌 참여 예측
• 파리의 문화적 강점 극대화 기대

11. 경제적 영향 예측
• 예상 관광 수입: 30억 유로
• 일자리 창출: 약 150,000개
• TV 중계권 수익: 약 15억 달러